In [1]:
import sys
sys.path.append('../ry')
from libry import *
from numpy import *

In [2]:
K = Config()
K.addFile("../rai-robotModels/pr2/pr2.g")
K.addFile("../models/tables.g")

K.addFrame("obj0", "table1", "type:ssBox size:[.1 .1 .2 .02] color:[1. 0. 0.], contact, logical={ object }, joint:rigid, Q:<t(0 0 .15)>" )
K.addFrame("obj1", "table1", "type:ssBox size:[.1 .1 .2 .02] color:[1. 0. 0.], contact, logical={ object }, joint:rigid, Q:<t(0 .2 .15)>" )
K.addFrame("obj2", "table1", "type:ssBox size:[.1 .1 .2 .02] color:[1. 0. 0.], contact, logical={ object }, joint:rigid, Q:<t(0 .4 .15)>" )
K.addFrame("obj3", "table1", "type:ssBox size:[.1 .1 .2 .02] color:[1. 0. 0.], contact, logical={ object }, joint:rigid, Q:<t(0 .6.15)>" )
K.addFrame("tray", "table2", "type:ssBox size:[.15 .15 .04 .02] color:[0. 1. 0.], logical={ table }, Q:<t(0 0 .07)>" );
K.addFrame("", "tray", "type:ssBox size:[.27 .27 .04 .02] color:[0. 1. 0.]" )
V = K.view()

In [3]:
lgp = K.lgp("../models/fol-pickAndPlace.g")
lgp.nodeInfo()

{'boundsConstraints': [0.0, 0.0, 0.0, 0.0, 0.0],
 'boundsCost': [0.0, 0.0, 0.0, 0.0, 0.0],
 'boundsFeasible': [1, 1, 1, 1, 1],
 'decision': '<ROOT>',
 'path': '',
 'state': ' STATE_0(START_STATE) {(gripper pr2R), (gripper pr2L), (table table2), (object obj0), (object obj1), (object obj2), (object obj3), (table tray), (partOf table2 tray), (on obj0 table1), (on obj1 table1), (on obj2 table1), (on obj3 table1), }'}

In [4]:
lgp.walkToNode("(grasp pr2R obj0) (grasp pr2L obj1) (place pr2R obj0 tray)")
lgp.nodeInfo()

{'boundsConstraints': [0.0, 0.0, 0.0, 0.0, 0.0],
 'boundsCost': [0.30000000000000004, 0.0, 0.0, 0.0, 0.0],
 'boundsFeasible': [1, 1, 1, 1, 1],
 'decision': '(place pr2R obj0 tray)',
 'path': '(grasp pr2R obj0) (grasp pr2L obj1) (place pr2R obj0 tray) ',
 'state': ' STATE_15(STATE_9) {(gripper pr2R), (gripper pr2L), (table table2), (object obj0), (object obj1), (object obj2), (object obj3), (table tray), (partOf table2 tray), (on obj0 table1), (on obj1 table1), (on obj2 table1), (on obj3 table1), (grasped pr2L obj1), (held obj1), (busy pr2L), (stable pr2L obj1),  decision(place pr2R obj0 tray), (placed obj0 tray), (on obj0 tray),  komo(touch pr2R obj0), (notheld obj0),  block(INFEASIBLE grasp ANY obj0),  komo(above obj0 tray), (stableOn tray obj0), }'}

In [5]:
lgp.optBound(BT.seq, True)
lgp.nodeInfo()

{'boundsConstraints': [0.0, 0.0, 0.2637363650645941, 0.0, 0.0],
 'boundsCost': [0.30000000000000004, 0.0, 0.3125876247331927, 0.0, 0.0],
 'boundsFeasible': [1, 1, 1, 1, 1],
 'decision': '(place pr2R obj0 tray)',
 'path': '(grasp pr2R obj0) (grasp pr2L obj1) (place pr2R obj0 tray) ',
 'state': ' STATE_15(STATE_9) {(gripper pr2R), (gripper pr2L), (table table2), (object obj0), (object obj1), (object obj2), (object obj3), (table tray), (partOf table2 tray), (on obj0 table1), (on obj1 table1), (on obj2 table1), (on obj3 table1), (grasped pr2L obj1), (held obj1), (busy pr2L), (stable pr2L obj1),  decision(place pr2R obj0 tray), (placed obj0 tray), (on obj0 tray),  komo(touch pr2R obj0), (notheld obj0),  block(INFEASIBLE grasp ANY obj0),  komo(above obj0 tray), (stableOn tray obj0), }'}

In [6]:
lgp.optBound(BT.path, True)
lgp.nodeInfo()

{'boundsConstraints': [0.0,
  0.0,
  0.2637363650645941,
  0.011257533502769073,
  0.0],
 'boundsCost': [0.30000000000000004,
  0.0,
  0.3125876247331927,
  2.2883087635607264,
  0.0],
 'boundsFeasible': [1, 1, 1, 1, 1],
 'decision': '(place pr2R obj0 tray)',
 'path': '(grasp pr2R obj0) (grasp pr2L obj1) (place pr2R obj0 tray) ',
 'state': ' STATE_15(STATE_9) {(gripper pr2R), (gripper pr2L), (table table2), (object obj0), (object obj1), (object obj2), (object obj3), (table tray), (partOf table2 tray), (on obj0 table1), (on obj1 table1), (on obj2 table1), (on obj3 table1), (grasped pr2L obj1), (held obj1), (busy pr2L), (stable pr2L obj1),  decision(place pr2R obj0 tray), (placed obj0 tray), (on obj0 tray),  komo(touch pr2R obj0), (notheld obj0),  block(INFEASIBLE grasp ANY obj0),  komo(above obj0 tray), (stableOn tray obj0), }'}

In [4]:
print("CHECK THE CONSOLE FOR OUTPUT -- THIS IS GENERATING LOTS OF FILES")
lgp.addTerminalRule("(on obj0 tray) (on obj1 tray) (on obj2 tray)")
lgp.run(2)

CHECK THE CONSOLE FOR OUTPUT -- THIS IS GENERATING LOTS OF FILES


In [ ]:
lgp=0